# Tohoku tsunami inversion
## Inversion using continuous adjoint

In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import scipy
from time import clock

In [2]:
from thetis import *

In [3]:
from adapt_utils.case_studies.tohoku.options import TohokuOptions
from adapt_utils.norms import total_variation
from adapt_utils.misc import gaussian, ellipse

In this hacky implementation we use some global variables

In [4]:
global level
level = 0

global op
op = TohokuOptions(level=0)
op.gauges.pop('21418')  # This pressure gauge isn't within the domain

global mesh
mesh = op.default_mesh

global diffs
diffs = [None for i in range(int(op.end_time/op.dt))]

global boundary_conditions
boundary_conditions = {
    100: ['freeslip', 'dirichlet'],
    200: ['freeslip'],
    300: ['freeslip'],
}

global basis_centres
basis_centres = [(0.7e+06, 4.2e+06), ]

### Forward model

As before, consider the shallow water equations linearised about lake at rest:

$$
    \frac{\partial\mathbf u}{\partial t}+g\nabla\eta+f\widehat{\mathbf z}\times\mathbf u=\boldsymbol0,\qquad
    \frac{\partial\eta}{\partial t}+\nabla\cdot(b\mathbf u)=0,\qquad
    \text{in}\quad(0,T]\times\Omega,
$$

equipped with boundary conditions

$$
    \mathbf u\cdot\widehat{\mathbf n}|_{\Gamma_{\mathrm{freeslip}}}\equiv0,\qquad
    \eta|_{\Gamma_D}\equiv0.
$$

Assume zero initial velocity and expand the initial surface over a radial basis:

$$
    \mathbf u|_{t=0}\equiv\boldsymbol0,\qquad
    \eta|_{t=0}=\sum_{k=1}^N m_kg_k,
$$

where $m_k$ are (constant in space) control parameters and $g_k$ are Gaussians.

In [5]:
def get_fields(fs):
    elev_space = fs.sub(1)
    b = Function(elev_space, name="Bathymetry (from GEBCO)").assign(op.set_bathymetry(elev_space))
    f = Function(elev_space, name="Coriolis parameter").assign(op.set_coriolis(elev_space))
    g = Constant(op.g)
    return b, f, g

### Quantity of Interest

Again, we have the QoI

$$
J(\mathbf u,\eta)=\frac12\sum_{g\in\mathcal G}\int_0^T\int_\Omega\mathbb1_g\big(\:\eta(x,t)-\eta_g(t)\:\big)^2\;\mathrm dx\;\mathrm dt,
$$

where $\mathbb1_g$ is an indicator function related to a sufficiently small neighbourhood of gauge $g$.

In [6]:
def sampled_timeseries(g, sample=1):
    time_prev = 0.0
    num_lines = sum(1 for line in open('resources/gauges/{:s}.dat'.format(g), 'r'))
    t, d, running = [], [], []
    with open('resources/gauges/{:s}.dat'.format(g), 'r') as f:
        for i in range(num_lines):
            time, dat = f.readline().split()
            time, dat = float(time), float(dat)
            if np.isnan(dat):
                continue
            running.append(dat)
            if i % sample == 0 and i > 0:
                t.append(0.5*(time + time_prev))
                d.append(np.mean(running))
                running = 0
                time_prev = time
                running = []
    return scipy.interpolate.interp1d(t, d, bounds_error=False, fill_value='extrapolate')

In [7]:
def get_basis_functions(fs, xy_skew=(48e+03, 96e+03), angle=pi/12):
    basis_functions = []
    for loc in basis_centres:
        basis_function = Function(fs)
        psi, phi = basis_function.split()
        phi.interpolate(gaussian([loc + xy_skew, ], fs.mesh(), rotation=angle))
        basis_functions.append(basis_function)
    return basis_functions

In [8]:
def solve_forward(ctrl_dat, level=0, gradient=False):
    print("FORWARD SOLVE FOR GRADIENT CALCULATION" if gradient else "FORWARD SOLVE")
    if isinstance(ctrl_dat, float):
        ctrl_dat = [ctrl_dat, ]
    
    # --- Setup function spaces
    
    fs = VectorFunctionSpace(mesh, "CG", 2)*FunctionSpace(mesh, "CG", 1)
    P0 = FunctionSpace(mesh, "DG", 0)
    
    # -- Setup fields
    
    b, f, g = get_fields(fs)
    
    # --- Setup PDE
    
    dtc = Constant(op.dt)
    
    u, eta = TrialFunctions(fs)
    z, zeta = TestFunctions(fs)

    q_ = Function(fs)
    u_, eta_ = q_.split()

    a = inner(z, u)*dx + inner(zeta, eta)*dx
    L = inner(z, u_)*dx + inner(zeta, eta_)*dx
    
    n = FacetNormal(mesh)

    def G(uv, elev):
        F = g*inner(z, grad(elev))*dx
        F += f*inner(z, as_vector((-uv[1], uv[0])))*dx
        F += -inner(grad(zeta), b*uv)*dx
        for tag in boundary_conditions:
            if 'freeslip' not in boundary_conditions[tag]:
                F += inner(zeta*n, b*uv)*ds(tag)
        return F

    a += 0.5*dtc*G(u, eta)
    L += -0.5*dtc*G(u_, eta_)

    q = Function(fs)
    u, eta = q.split()

    bcs = []
    for tag in boundary_conditions:
        if 'dirichlet' in boundary_conditions[tag]:
            bcs.append(DirichletBC(fs.sub(1), 0, tag))

    params = {
        "snes_type": "ksponly",
        "ksp_type": "gmres",
        "pc_type": "fieldsplit",
        "pc_fieldsplit_type": "multiplicative",
    }

    problem = LinearVariationalProblem(a, L, q, bcs=bcs)
    solver = LinearVariationalSolver(problem, solver_parameters=params)
    
    # --- Setup initial condition / control
    
    basis_functions = get_basis_functions(fs)
    try:
        assert len(basis_functions) == len(ctrl_dat)
    except AssertionError:
        msg = "Number of basis functions and number of basis coefficients do not match ({:d} vs {:d})"
        raise ValueError(msg.format(len(basis_functions), len(ctrl_dat)))
    eta_.project(sum(m_i*phi_i.split()[1] for m_i, phi_i in zip(ctrl_dat, basis_functions)))
    
    # --- Setup QoI
    
    radius = 20.0e+03*pow(0.5, level)  # The finer the mesh, the more precise the indicator region
    for gauge in op.gauges:
        op.gauges[gauge]['indicator'] = interpolate(ellipse([op.gauges[gauge]["coords"] + (radius,), ], mesh), P0)
        op.gauges[gauge]['interpolator'] = sampled_timeseries(gauge, sample=60 if gauge[0] == 'P' else 1)
        
    op.end_time = 1440.0
        
    times = []

    # --- Time integrate
    
    t = 0.0
    iteration = 0
    J = 0
    eta_obs = Constant(0.0)
    weight = Constant(1.0)
    tic = clock()
    while t < op.end_time:
        times.append(t)
        if iteration % 48 == 0:
            toc = clock() - tic
            print("    simulation time {:4.1f} minutes    wallclock time {:4.1f} seconds".format(t/60, toc))
            tic = clock()

        # Solve forward equation at current timestep
        solver.solve()

        # Time integrate QoI
        weight.assign(0.5 if np.allclose(t, 0.0) or t >= op.end_time - 0.5*op.dt else 1.0)
        u, eta = q.split()
        for gauge in op.gauges:
            indicator = op.gauges[gauge]['indicator']

            # Interpolate observations
            obs = float(op.gauges[gauge]['interpolator'](t))
            eta_obs.assign(obs)

            # Continuous form of error
            J += assemble(weight*dtc*0.5*indicator*(eta - eta_obs)**2*dx)
            
            # Save RHS of adjoint equation
            diffs[iteration] = interpolate(indicator*(eta - eta_obs), fs.sub(1))  # TODO: Checkpoint to disk

        # Increment
        q_.assign(q)
        t += op.dt
        iteration += 1

    assert np.allclose(t, op.end_time), print("mismatching end time ({:.2f} vs {:.2f})".format(t, op.end_time))
    toc = clock() - tic
    print("    simulation time {:4.1f} minutes    wallclock time {:4.1f} seconds".format(t/60, toc))
    print("    Quantity of interest = {:.8e}".format(J))
    return J

In [9]:
def reduced_functional(m):
    return solve_forward(m)

In [10]:
print("Quantity of interest = {:.8e}".format(reduced_functional(1.0)))

FORWARD SOLVE
    simulation time  0.0 minutes    wallclock time  0.0 seconds
    simulation time  4.0 minutes    wallclock time 26.2 seconds
    simulation time  8.0 minutes    wallclock time 26.5 seconds
    simulation time 12.0 minutes    wallclock time 26.2 seconds
    simulation time 16.0 minutes    wallclock time 26.7 seconds
    simulation time 20.0 minutes    wallclock time 26.6 seconds
    simulation time 24.0 minutes    wallclock time 26.6 seconds
    Quantity of interest = 6.85574698e+13
Quantity of interest = 6.85574698e+13


### Adjoint problem

The adjoint problem is comprised of the adjoint equations

$$
    -\frac{\partial\mathbf u^*}{\partial t}-b\nabla\eta^*-f\widehat{\mathbf z}\times\mathbf u^*=\boldsymbol0,\qquad
    -\frac{\partial\eta^*}{\partial t}-g\nabla\cdot u^*=\sum_{g\in\mathcal G}\mathbb 1_g(\eta-\eta_g),\qquad
    \text{in}\quad(0,T]\times\Omega,
$$

(using the expressions for $\frac{\partial J}{\partial\mathbf u}$ and $\frac{\partial J}{\partial\eta}$ stated earlier) and the final time and boundary conditions

$$
    \mathbf u^*\cdot\widehat{\mathbf n}|_{\partial\Omega\backslash\Gamma_D}\equiv0,\qquad
    \eta^*|_{\partial\Omega\backslash\Gamma_{\mathrm{freeslip}}}\equiv0,\qquad
    \mathbf u^*|_{t=T}\equiv\boldsymbol0,\qquad
    \eta^*|_{t=T}\equiv0.
$$

Recall that in our case $\partial\Omega\backslash\Gamma_{\mathrm{freeslip}}=\emptyset$ and $\partial\Omega\backslash\Gamma_D$ corresponds to coastal boundary tags 200 and 300.

In [10]:
def solve_adjoint(gradient=True):
    print("ADJOINT SOLVE FOR GRADIENT CALCULATION" if gradient else "ADJOINT SOLVE")
    
    # --- Setup function spaces
    
    fs = VectorFunctionSpace(mesh, "CG", 2)*FunctionSpace(mesh, "CG", 1)
    P0 = FunctionSpace(mesh, "DG", 0)
    
    # -- Setup fields
    
    b, f, g = get_fields(fs)
    
    # --- Setup PDE

    dtc = Constant(op.dt)
    
    u_star, eta_star = TrialFunctions(fs)
    z, zeta = TestFunctions(fs)
    q_star_ = Function(fs)
    u_star_, eta_star_ = q_star_.split()

    a = -inner(z, u_star)*dx - inner(zeta, eta_star)*dx
    L = -inner(z, u_star_)*dx - inner(zeta, eta_star_)*dx

    n = FacetNormal(mesh)

    def G(uv_star, elev_star):
        F = -b*inner(z, grad(elev_star))*dx
        F += -f*inner(z, as_vector((-uv_star[1], uv_star[0])))*dx
        F += g*inner(grad(zeta), uv_star)*dx
        for tag in boundary_conditions:
            if 'dirichlet' not in boundary_conditions[tag]:
                F += -inner(zeta*n, uv_star)*ds(tag)
        return F

    a += 0.5*dtc*G(u_star, eta_star)
    L += -0.5*dtc*G(u_star_, eta_star_)

    rhs = Function(fs.sub(1))

    L += zeta*rhs*dx

    q_star = Function(fs)
    u_star, eta_star = q_star.split()

    # bc = DirichletBC(fs.sub(1), 0, [200, 300])
    bc = None
    bcs = []
    for tag in boundary_conditions:
        if 'freeslip' not in boundary_conditions[tag]:
            bcs.append(DirichletBC(fs.sub(1), 0, tag))

    params = {
        "snes_type": "ksponly",
        "ksp_type": "gmres",
        "pc_type": "fieldsplit",
        "pc_fieldsplit_type": "multiplicative",
    }

    problem = LinearVariationalProblem(a, L, q_star, bcs=bc)
    solver = LinearVariationalSolver(problem, solver_parameters=params)
    
    # --- Setup indicators
    
    radius = 20.0e+03*pow(0.5, level)  # The finer the mesh, the more precise the indicator region
    for gauge in op.gauges:
        op.gauges[gauge]['indicator'] = interpolate(ellipse([op.gauges[gauge]["coords"] + (radius,), ], mesh), P0)
    
    # --- Time integrate
    
    t = op.end_time
    iteration = int(op.end_time/op.dt)
    tic = clock()
    while t > 0.0:
        if iteration % 48 == 0:
            toc = clock() - tic
            print("    simulation time {:4.1f} minutes    wallclock time {:4.1f} seconds".format(t/60, toc))
            tic = clock()

        # Evaluate function appearing in RHS
        assert diffs[iteration-1] is not None
        rhs.interpolate(diffs[iteration-1])
        diffs[iteration-1] = None  # Reset saved data

        # Solve adjoint equation at current timestep
        solver.solve()

        # Increment
        q_star_.assign(q_star)
        t -= op.dt
        iteration -= 1

    assert np.allclose(t, 0.0)
    toc = clock() - tic
    print("    simulation time {:4.1f} minutes    wallclock time {:4.1f} seconds".format(t/60, toc))
    return q_star

Further, recall that the gradient may be computed as

$$
    \frac{\mathrm dJ}{\mathrm dm_k} = \int_\Omega g_k\eta^*|_{t=0}\;\mathrm dx.
$$

In [11]:
def gradient(m):
    if diffs[0] is None:
        J = solve_forward(m, level=level, gradient=True)
    adj_sol0 = solve_adjoint()
    adj_sol0 *= -1  # FIXME
    basis_functions = get_basis_functions(adj_sol0.function_space())
    g = np.array([assemble(inner(adj_sol0, phi)*dx) for phi in basis_functions])
    print("Gradient = {:.4e}".format(g[0]))
    return g

In [13]:
gradient(1.0)

ADJOINT SOLVE FOR GRADIENT CALCULATION
    simulation time 24.0 minutes    wallclock time  0.0 seconds
    simulation time 20.0 minutes    wallclock time 12.6 seconds
    simulation time 16.0 minutes    wallclock time 13.5 seconds
    simulation time 12.0 minutes    wallclock time 12.6 seconds
    simulation time  8.0 minutes    wallclock time 13.0 seconds
    simulation time  4.0 minutes    wallclock time 13.6 seconds
    simulation time  0.0 minutes    wallclock time 12.9 seconds
Gradient = -4.3823e+08


array([-4.38226764e+08])

# TODO: Fix continuous adjoint above

Now that we have functions for solving the forward problem and computing the gradient, we can apply a quasi-Newton optimisation method such as BFGS to optimise the control parameter.
BFGS may be expressed in pseudocode as

    m_0 := <initial guess>
    i := 0
    I := <identity matrix>
    while not converged:
        J_i := solve_forward(m_i)     # current quantity of interest value
        g_i := compute_gradient(m_i)  # current value of gradient
        
        # Approximate Hessian
        if i == 0:
            H_i := I
        else:
            s := m_i - m_{i-1}
            y := g_i - g_{i-1}
            H_i := (I - outer(s,y)/inner(y,s)) * H_{i-1} * (I - outer(y,s)/inner(y,s)) + outer(s,s)/inner(y,s)
            
        p_i := H_i*g_i  # search direction
        alpha_i := <step length computed using a line search>
        m_{i+1} := m_i + alpha_i*p_i
        
        <check convergence>
        
        i += 1
        
where here `m_i` is a vector of control parameters.

In [12]:
m_init = np.array([1.0, ])
opt_parameters = {
    'tol': 1.0e-03,
    'options': {
#         'maxiter': 5,
        'disp': True,
        'gtol': 1.0,
    },
    'method': 'BFGS',
}

def opt_cb(m_current):
    print("LINE SEARCH COMPLETE\nCurrent control value = {:.8e}\n".format(*m_current))

opt_res = scipy.optimize.minimize(reduced_functional, m_init, jac=gradient, callback=opt_cb, **opt_parameters)

FORWARD SOLVE
    simulation time  0.0 minutes    wallclock time  0.0 seconds
    simulation time  4.0 minutes    wallclock time 26.5 seconds
    simulation time  8.0 minutes    wallclock time 26.7 seconds
    simulation time 12.0 minutes    wallclock time 27.1 seconds
    simulation time 16.0 minutes    wallclock time 27.4 seconds
    simulation time 20.0 minutes    wallclock time 27.2 seconds
    simulation time 24.0 minutes    wallclock time 27.5 seconds
    Quantity of interest = 6.85574698e+13
ADJOINT SOLVE FOR GRADIENT CALCULATION
    simulation time 24.0 minutes    wallclock time  0.0 seconds
    simulation time 20.0 minutes    wallclock time 12.6 seconds
    simulation time 16.0 minutes    wallclock time 12.8 seconds
    simulation time 12.0 minutes    wallclock time 12.9 seconds
    simulation time  8.0 minutes    wallclock time 12.9 seconds
    simulation time  4.0 minutes    wallclock time 13.0 seconds
    simulation time  0.0 minutes    wallclock time 13.0 seconds
Gradient 

In [13]:
assert opt_res.success  # check for successful termination
m_opt = opt_res.x  # result of optimisation
print("Optimised control parameter = {:.4e}".format(*m_opt))

Optimised control parameter = 3.2676e+00
